In [1]:
"""A very simple MNIST classifier.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/beginners
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data


import keras
from keras.layers.core import Dense, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

from keras import backend as K


K.image_data_format() 

/Users/cuiyue/anaconda3/envs/py27/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


'channels_last'

In [2]:
# Import data
data_dir = '/Users/cuiyue/Desktop/AI/第六周/mnist'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

# Define loss and optimizer
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
learning_rate = tf.placeholder(tf.float32)

with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])


net = Conv2D(64, kernel_size=[4, 4], strides=[1,1],activation='relu',
                 padding='same',input_shape=[28,28,1], kernel_initializer= keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None),
             bias_initializer=  keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None))(x_image)
net = MaxPooling2D(pool_size=[2,2])(net)
net = Conv2D(128, kernel_size=[4, 4], strides=[1,1],activation='relu',
                padding='same', kernel_initializer= keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None), 
             bias_initializer=  keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None))(net)
net = MaxPooling2D(pool_size=[2,2])(net)
net = Flatten()(net)
net = Dense(2000, activation='relu', kernel_initializer= keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None), 
            bias_initializer= keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None))(net)
net = Dense(10,activation='softmax', kernel_initializer= keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None), 
            bias_initializer=  keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None))(net)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/cuiyue/Desktop/AI/第六周/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/cuiyue/Desktop/AI/第六周/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /Users/cuiyue/Desktop/AI/第六周/mnist/t10k-images-idx3-ubyte.gz
Extracting /Users/cuiyue/Desktop/AI/第六周/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# 定义动态学习率
init_learning_rate = tf.placeholder(tf.float32)
epoch_steps = tf.to_int64(tf.div(60000, tf.shape(x)[0]))
global_step = tf.train.get_or_create_global_step()
current_epoch = global_step//epoch_steps
decay_times = current_epoch 
current_learning_rate = tf.multiply(init_learning_rate, 
                                    tf.pow(0.575, tf.to_float(decay_times)))

In [4]:
from keras.objectives import categorical_crossentropy
cross_entropy = tf.reduce_mean(categorical_crossentropy(y_, net))

In [5]:
l2_loss = tf.add_n( [tf.nn.l2_loss(w) for w in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)] )
total_loss = cross_entropy + 7e-5*l2_loss

train_step = tf.train.GradientDescentOptimizer(current_learning_rate).minimize(total_loss)

sess = tf.Session()

In [6]:
K.set_session(sess)

In [7]:
init_op = tf.global_variables_initializer()
sess.run(init_op)
# Train
for step in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    lr = 0.01
    _, loss, l2_loss_value, total_loss_value,  current_lr_value= sess.run(
               [train_step, cross_entropy, l2_loss, total_loss, current_learning_rate], 
               feed_dict={x: batch_xs, y_: batch_ys, init_learning_rate:lr})
  
    if (step+1) % 100 == 0:
        print('step %d, entropy loss: %f, l2_loss: %f, total loss: %f' % 
            (step+1, loss, l2_loss_value, total_loss_value))
        # Test trained model
        correct_prediction = tf.equal(tf.argmax(net, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print(sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys}))
    if (step+1) % 1000 == 0:
        print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                    y_: mnist.test.labels}))

step 100, entropy loss: 0.523903, l2_loss: 12288.213867, total loss: 1.384079
0.91
step 200, entropy loss: 0.596091, l2_loss: 12287.519531, total loss: 1.456218
0.83
step 300, entropy loss: 0.347491, l2_loss: 12286.343750, total loss: 1.207535
0.95
step 400, entropy loss: 0.191921, l2_loss: 12285.023438, total loss: 1.051872
0.95
step 500, entropy loss: 0.133618, l2_loss: 12283.667969, total loss: 0.993475
0.98
step 600, entropy loss: 0.177202, l2_loss: 12282.270508, total loss: 1.036961
0.95
step 700, entropy loss: 0.113485, l2_loss: 12280.831055, total loss: 0.973143
0.99
step 800, entropy loss: 0.153801, l2_loss: 12279.405273, total loss: 1.013359
0.95
step 900, entropy loss: 0.052813, l2_loss: 12277.944336, total loss: 0.912269
1.0
step 1000, entropy loss: 0.146940, l2_loss: 12276.464844, total loss: 1.006293
0.97
0.9586
step 1100, entropy loss: 0.151878, l2_loss: 12274.969727, total loss: 1.011125
0.98
step 1200, entropy loss: 0.041367, l2_loss: 12273.504883, total loss: 0.900512
